In [1]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu

In [2]:
import pandas as pd
import numpy as np

from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.document_loaders import DataFrameLoader

/Users/quentinfisch/Documents/EPITA/ING3/SCIA/S9/ALEIA/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('data.csv', sep=';')
df

,ID_AIDE,Collectivité,Code SIREN de la collectivité,Référence administrative,Nom de l'aide / de la démarche,Titre alternatif de l'aide,Chapô (HTML),Chapô (TXT),Date,Thème,...,Infos pratiques - Contact (HTML),Infos pratiques - Contact (TXT),Engagements du bénéficiaire,Date ouverture,Date clôture,Délibération cadre,Lien vers le portail iledefrance.fr,Démarche en ligne,DATEDEBUTFUTURCAMPAGNE,qui
0,HJlWyZsxp,Région Île-de-France,237500079,646,RÊV’ELLES : Programme « RVL Découverte »,Olympe - Rêv’Elles : Programme « RVL Découverte »,"<p style=""margin-bottom:0cm;text-align:left;"">...",Ce programme de la Région a pour ambition de f...,2023-10-17T21:30:05+02:00,Éducation & recherche : Lycée,...,"<p><span style=""font-family:CIDFont+F2;"">Marie...","Marie Castanet, responsable des programmes et ...",NaN,2023-09-01T00:00:00+02:00,2024-06-30T00:00:00+02:00,NaN,https://www.iledefrance.fr/aides-et-appels-a-p...,False,NaN,Établissement d'enseignement secondaire|||Part...
1,rkZe_5clT,Région Île-de-France,237500079,557,Appel à projets Actions de formation en situat...,Appel à projets Actions de formation en situat...,"<p style=""margin-bottom:0cm;line-height:normal...",La Région soutient des actions de formation in...,2023-10-04T21:30:05+02:00,Emploi & formation : Formation professionnelle...,...,"<p><a href=""mailto:aap.afest@iledefrance.fr"" t...",aap.afest@iledefrance.fr Les questions sur AA...,Charte des valeurs de la République et de la l...,2022-07-07T00:00:00+02:00,2022-09-30T23:59:00+02:00,NaN,https://www.iledefrance.fr/aides-et-appels-a-p...,False,NaN,"Établissement ou organismes de formation (OF, ..."
2,r1gaxD2xa,Région Île-de-France,237500079,114,Aide aux projets œuvrant à la valorisation des...,NaN,Actions visant à valoriser et mieux faire conn...,Actions visant à valoriser et mieux faire conn...,2023-10-05T21:30:10+02:00,Culture : Patrimoine,...,"Chargée de mission pour les départements 75, 7...","Chargée de mission pour les départements 75, 7...","Mesure ""Trouvez un stage"",Charte des valeurs d...",NaN,NaN,CR 2017-084 modifié par CR 2020-015,https://www.iledefrance.fr/aides-et-appels-a-p...,False,NaN,Professionnel - Patrimoine|||Professionnel - C...
3,rJWfsdW-T,Région Île-de-France,237500079,23,"Parentalité, ouverture socio-culturelle et sco...","Parentalité, ouverture socio-culturelle et sco...","Dans le cadre de son programme<a href="" https:...",Dans le cadre de son programme Région solidair...,2023-10-09T21:30:37+02:00,Santé & Social : Action sociale,...,"<a href=""mailto:solidarites@iledefrance.fr"">so...",solidarites@iledefrance.fr,"Mesure ""Trouvez un stage"",Charte des valeurs d...",2023-03-20T00:00:00+01:00,2023-05-09T00:00:00+02:00,CP 2018-63,https://www.iledefrance.fr/aides-et-appels-a-p...,False,NaN,Association - Régie par la loi de 1901
4,ByiJ6L2g6,Région Île-de-France,237500079,112,Soutien à la restauration et à l’aménagement d...,NaN,La Région contribue à préserver et favoriser l...,La Région contribue à préserver et favoriser l...,2023-10-24T17:00:05+02:00,Culture : Patrimoine,...,<p>Chargée de mission pour les départements 75...,"Chargée de mission pour les départements 75, 7...","Mesure ""Trouvez un stage"",Charte des valeurs d...",NaN,NaN,CR 2017-084 modifié par CR 2020-015,https://www.iledefrance.fr/aides-et-appels-a-p...,False,NaN,Association - Fondation|||Association - ONG|||...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,ryJ7J8Af6,Région Île-de-France,237500079,338,CONVENTIONS DE DEVELOPPEMENT CULTUREL - ARTS V...,Conventions de développement culturel Arts vis...,<p><span>La Région accompagne à travers les co...,La Région accompagne à travers les conventions...,2023-10-31T11:00:09+01:00,Culture : Arts plastiques; numériques et urbains,...,"<p><span style=""font-family:Verdana, sans-seri...","Direction de la culture Edouard Brena, chargé ...",Charte des valeurs de la République et de la l...,2023-10-18T00:00:00+02:00,2023-11-22T00:00:00+01:00,CR 2017-76 DU 

In [4]:
df.columns

Index(['ID_AIDE', 'Collectivité', 'Code SIREN de la collectivité',
       'Référence administrative', 'Nom de l'aide / de la démarche',
       'Titre alternatif de l'aide', 'Chapô (HTML)', 'Chapô (TXT)', 'Date',
       'Thème', 'Mots clés', 'Pour quel type de projet ?',
       'Pour quel type de projet ? (TXT)', 'Qui peut en bénéficier ?',
       'Précisions sur le public bénéficiaire (HTML)',
       'Précisions sur le public bénéficiaire (TXT)',
       'Quelle est la nature de l'aide ?',
       'Quelle est la nature de l'aide ? (TXT)', 'Quelles démarches ?',
       'Quelles démarches ? (TXT)', 'À noter', 'À noter (TXT)',
       'Documents informatifs', 'Documents nécessaires',
       'Infos pratiques - Contact (HTML)', 'Infos pratiques - Contact (TXT)',
       'Engagements du bénéficiaire', 'Date ouverture', 'Date clôture',
       'Délibération cadre', 'Lien vers le portail iledefrance.fr',
       'Démarche en ligne', 'DATEDEBUTFUTURCAMPAGNE', 'qui'],
      dtype='object')

In [5]:
columns_to_drop = ["ID_AIDE", "Collectivité", "Code SIREN de la collectivité", "Référence administrative", "Chapô (HTML)", "Pour quel type de projet ?", "Précisions sur le public bénéficiaire (HTML)", "Quelle est la nature de l'aide ?", "Quelles démarches ?", "À noter", "Documents informatifs", "Documents nécessaires", "Infos pratiques - Contact (HTML)", "Date", "Délibération cadre", "DATEDEBUTFUTURCAMPAGNE"]
df = df.drop(columns=columns_to_drop)
df

,Nom de l'aide / de la démarche,Titre alternatif de l'aide,Chapô (TXT),Thème,Mots clés,Pour quel type de projet ? (TXT),Qui peut en bénéficier ?,Précisions sur le public bénéficiaire (TXT),Quelle est la nature de l'aide ? (TXT),Quelles démarches ? (TXT),À noter (TXT),Infos pratiques - Contact (TXT),Engagements du bénéficiaire,Date ouverture,Date clôture,Lien vers le portail iledefrance.fr,Démarche en ligne,qui
0,RÊV’ELLES : Programme « RVL Découverte »,Olympe - Rêv’Elles : Programme « RVL Découverte »,Ce programme de la Région a pour ambition de f...,Éducation & recherche : Lycée,Egalité femmes/hommes,RVL Découverte fait découvrir les différents m...,"Établissement d'enseignement secondaire,Partic...",Les jeunes femmes des lycées francilien.,Action gratuite pour les lycées franciliens.,"Pour bénéficier de ce programme, s’inscrire di...",NaN,"Marie Castanet, responsable des programmes et ...",NaN,2023-09-01T00:00:00+02:00,2024-06-30T00:00:00+02:00,https://www.iledefrance.fr/aides-et-appels-a-p...,False,Établissement d'enseignement secondaire|||Part...
1,Appel à projets Actions de formation en situat...,Appel à projets Actions de formation en situat...,La Région soutient des actions de formation in...,Emploi & formation : Formation professionnelle...,Emploi,Les projets soutenus sont des actions de forma...,"Établissement ou organismes de formation (OF, ...","Les porteurs éligibles sont : Les entreprises,...",Le taux d'intervention maximum sur le projet s...,Les dossiers devront être déposés sur mesdemar...,NaN,aap.afest@iledefrance.fr Les questions sur AA...,Charte des valeurs de la République et de la l...,2022-07-07T00:00:00+02:00,2022-09-30T23:59:00+02:00,https://www.iledefrance.fr/aides-et-appels-a-p...,False,"Établissement ou organismes de formation (OF, ..."
2,Aide aux projets œuvrant à la valorisation des...,NaN,Actions visant à valoriser et mieux faire conn...,Culture : Patrimoine,"Patrimoine,Attractivité",Actions de valorisation (expos physiques ou vi...,"Professionnel - Patrimoine,Professionnel - Cul...",Sont éligibles les propriétaires publics et pr...,La subvention est calculée par application d’u...,Les demandes d’aide devront être déposées sur ...,Ouverture permanente,"Chargée de mission pour les départements 75, 7...","Mesure ""Trouvez un stage"",Charte des valeurs d...",NaN,NaN,https://www.iledefrance.fr/aides-et-appels-a-p...,False,Professionnel - Patrimoine|||Professionnel - C...
3,"Parentalité, ouverture socio-culturelle et sco...","Parentalité, ouverture socio-culturelle et sco...",Dans le cadre de son programme Région solidair...,Santé & Social : Action sociale,Région solidaire,Les projets doivent correspondre à l’une ou pl...,Association - Régie par la loi de 1901,Associations de loi 1901 : l'association doit ...,Subvention en fonctionnement : la subvention r...,Dépôt du dossier de candidature sur la platefo...,Le bénéficiaire de toute subvention de la Régi...,solidarites@iledefrance.fr,"Mesure ""Trouvez un stage"",Charte des valeurs d...",2023-03-20T00:00:00+01:00,2023-05-09T00:00:00+02:00,https://www.iledefrance.fr/aides-et-appels-a-p...,False,Association - Régie par la loi de 1901
4,Soutien à la restauration et à l’aménagement d...,NaN,La Région contribue à préserver et favoriser l...,Culture : Patrimoine,"Patrimoine,Attractivité","Restauration du bâtiment (clos et couvert), Me...","Association - Fondation,Association - ONG,Asso...",Dispositif ouvert aux propriétaires publics ou...,La subvention jusqu'à 30 % des dépenses éligib...,Les demandes d’aide devront être déposées sur ...,Ouverture permanente,"Chargée de mission pour les départements 75, 7...","Mesure ""Trouvez un stage"",Charte des valeurs d...",NaN,NaN,https://www.iledefrance.fr/aides-et-appels-a-p...,False,Association - Fondation|||Association - ONG|||...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,CONVENTIONS DE DEVELOPPEMENT CULTUREL - ARTS V...,Conventions de développement culturel Arts vis...,La Région a

In [6]:
# add a new column which contacts (using newlines) the data from the column "Nom de l'aide / de la démarche", "TChapô (TXT)	" and "Mots clés"
df['text'] = "Nom de l'aide: " + df["Nom de l'aide / de la démarche"] + "\n" + "Chapo: " + df["Chapô (TXT)"] + "\n" + "Mots clés: " + df["Mots clés"]
df["text"]

0      Nom de l'aide: RÊV’ELLES : Programme « RVL Déc...
1      Nom de l'aide: Appel à projets Actions de form...
2      Nom de l'aide: Aide aux projets œuvrant à la v...
3      Nom de l'aide: Parentalité, ouverture socio-cu...
4      Nom de l'aide: Soutien à la restauration et à ...
                             ...                        
322    Nom de l'aide: CONVENTIONS DE DEVELOPPEMENT CU...
323    Nom de l'aide: PM'up Souveraineté Transition é...
324    Nom de l'aide: Fonds d'aide à la création de j...
325    Nom de l'aide: Soutien aux projets associatifs...
326    Nom de l'aide: Soutien aux commerces ruraux : ...
Name: text, Length: 327, dtype: object

In [7]:
df_reduced = df[["text", "qui", "Thème"]]
df_reduced

,text,qui,Thème
0,Nom de l'aide: RÊV’ELLES : Programme « RVL Déc...,Établissement d'enseignement secondaire|||Part...,Éducation & recherche : Lycée
1,Nom de l'aide: Appel à projets Actions de form...,"Établissement ou organismes de formation (OF, ...",Emploi & formation : Formation professionnelle...
2,Nom de l'aide: Aide aux projets œuvrant à la v...,Professionnel - Patrimoine|||Professionnel - C...,Culture : Patrimoine
3,"Nom de l'aide: Parentalité, ouverture socio-cu...",Association - Régie par la loi de 1901,Santé & Social : Action sociale
4,Nom de l'aide: Soutien à la restauration et à ...,Association - Fondation|||Association - ONG|||...,Culture : Patrimoine
...,...,...,...
322,Nom de l'aide: CONVENTIONS DE DEVELOPPEMENT CU...,Professionnel - Créateur d'entreprise|||Profes...,Culture : Arts plastiques; numériques et urbains
323,Nom de l'aide: PM'up Souveraineté Transition é...,Professionnel - TPE < 10|||Professionnel - PME...,"Territoire : SDRIF Environnemental,Économie : ..."
324,Nom de l'aide: Fonds d'aide à la création de j...,Professionnel - Créateur d'entreprise|||Profes...,Culture : Jeux vidéo
325,Nom de l'aide: Soutien aux projets associatifs...,Association - Régie par la loi de 1901,Santé & Social : Action sociale


In [8]:
loader = DataFrameLoader(df, page_content_column="Nom de l'aide / de la démarche")
data = loader.load()
data

[Document(page_content='RÊV’ELLES : Programme « RVL Découverte »', metadata={"Titre alternatif de l'aide": 'Olympe - Rêv’Elles : Programme « RVL Découverte »', 'Chapô (TXT)': 'Ce programme de la Région a pour ambition de faire découvrir aux jeunes filles un grand nombre de secteurs et de métiers avec des objectifs pédagogiques d’ouverture du champ des possibles, d’affranchissement des déterminismes et de pouvoir d’agir.', 'Thème': 'Éducation & recherche : Lycée', 'Mots clés': 'Egalité femmes/hommes', 'Pour quel type de projet ? (TXT)': "RVL Découverte fait découvrir les différents métiers et secteurs d’activités aux jeunes filles et les accompagne à dépasser stéréotypes de genre, difficultés de représentation des métiers, biais d’orientation et autocensure.Sensibilisation sur la déconstruction des croyances limitantes auprès d’établissements scolaires ou de structures jeunesse.Programme RVL Ton Potentiel : parcours intensif de 5 jours puis suivi de 5 mois avec coachings collectifs et i

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# 'data' holds the text you want to split, split the text into documents using the text splitter.
docs = text_splitter.split_documents(data)
docs[:2]

[Document(page_content='RÊV’ELLES : Programme « RVL Découverte »', metadata={"Titre alternatif de l'aide": 'Olympe - Rêv’Elles : Programme « RVL Découverte »', 'Chapô (TXT)': 'Ce programme de la Région a pour ambition de faire découvrir aux jeunes filles un grand nombre de secteurs et de métiers avec des objectifs pédagogiques d’ouverture du champ des possibles, d’affranchissement des déterminismes et de pouvoir d’agir.', 'Thème': 'Éducation & recherche : Lycée', 'Mots clés': 'Egalité femmes/hommes', 'Pour quel type de projet ? (TXT)': "RVL Découverte fait découvrir les différents métiers et secteurs d’activités aux jeunes filles et les accompagne à dépasser stéréotypes de genre, difficultés de représentation des métiers, biais d’orientation et autocensure.Sensibilisation sur la déconstruction des croyances limitantes auprès d’établissements scolaires ou de structures jeunesse.Programme RVL Ton Potentiel : parcours intensif de 5 jours puis suivi de 5 mois avec coachings collectifs et i

In [10]:
# use distilbert to embed the text
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

In [11]:
db = FAISS.from_documents(docs, embeddings)

In [12]:
question = "Mon logement tombe en ruines. Il faudrait le rénover. Est-ce qu'il y a une aide régionale pour ça?"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

Renouvellement des chauffages au fuel, au bois ou au charbon


### Méthode 2

In [13]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.vectorstores import Chroma
from langchain import hub
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
import os

load_dotenv()

True

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = True

In [15]:
loader = DataFrameLoader(df, page_content_column="Nom de l'aide / de la démarche")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)
docs[:2]

[Document(page_content='RÊV’ELLES : Programme « RVL Découverte »', metadata={"Titre alternatif de l'aide": 'Olympe - Rêv’Elles : Programme « RVL Découverte »', 'Chapô (TXT)': 'Ce programme de la Région a pour ambition de faire découvrir aux jeunes filles un grand nombre de secteurs et de métiers avec des objectifs pédagogiques d’ouverture du champ des possibles, d’affranchissement des déterminismes et de pouvoir d’agir.', 'Thème': 'Éducation & recherche : Lycée', 'Mots clés': 'Egalité femmes/hommes', 'Pour quel type de projet ? (TXT)': "RVL Découverte fait découvrir les différents métiers et secteurs d’activités aux jeunes filles et les accompagne à dépasser stéréotypes de genre, difficultés de représentation des métiers, biais d’orientation et autocensure.Sensibilisation sur la déconstruction des croyances limitantes auprès d’établissements scolaires ou de structures jeunesse.Programme RVL Ton Potentiel : parcours intensif de 5 jours puis suivi de 5 mois avec coachings collectifs et i

In [28]:
embeddings = OpenAIEmbeddings()
vector_store = Chroma.from_documents(docs, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 4}, search_type="similarity")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [29]:
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

template = """Voici les différentes aides que le client peut obtenir:
{context}

Tu dois répondre en détaillant les 2 aides qui te semblent les plus pertinentes pour le client.

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [20]:
print(rag_chain.invoke("Je veux passer à la voiture électrique"))

Réponse: Si le client souhaite passer à la voiture électrique, deux aides qui pourraient être particulièrement pertinentes sont le Plan route de demain et la transformation de véhicules thermiques en véhicules propres.

1. Plan route de demain : Électromobilité
Le Plan route de demain est une aide qui vise à encourager l'électromobilité en offrant des incitations financières pour l'achat d'un véhicule électrique neuf. Cette aide peut prendre la forme d'une prime à l'achat ou d'un bonus écologique, qui permet de réduire le coût d'acquisition d'un véhicule électrique. Cela peut être particulièrement intéressant pour le client, car cela lui permettrait de bénéficier d'une réduction significative sur le prix d'achat de sa nouvelle voiture électrique, rendant ainsi cette transition plus abordable financièrement.

2. Transformation de véhicules thermiques en véhicules propres
Si le client possède déjà un véhicule thermique, une autre aide pertinente pourrait être la transformation de ce véhi

In [30]:
print(rag_chain.invoke("Je veux acheter un food truck électrique"))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Réponse: Pour l'achat d'un food truck électrique, deux aides peuvent être particulièrement pertinentes pour le client:

1. Commerce ou artisanat ambulant : acquisition de véhicules propres
Cette aide est spécifiquement destinée aux commerçants ou artisans ambulants qui souhaitent acquérir un véhicule propre, tel qu'un food truck électrique. Elle peut prendre la forme d'une subvention ou d'une aide financière pour soutenir l'investissement initial. Cette aide permettrait au client de réduire les coûts liés à l'achat du food truck électrique, ce qui pourrait rendre le projet plus accessible financièrement.

2. Acquisition de véhicules propres par les professionnels franciliens
Cette aide s'adresse aux professionnels de la région francilienne qui souhaitent acquérir un véhicule propre, y compris les food trucks électriques. Elle peut prendre la forme d'une prime à l'achat ou d'une subvention pour encourager la transition vers des véhicules moins polluants. Cette aide permettrait au client

In [27]:
df[df["Nom de l\'aide / de la démarche"] == "Plan route de demain : Électromobilité"]["Quelles démarches ? (TXT)"].item()

'Déposez sur mesdemarches.iledefrance.fr votre dossier de demande de subvention présentant le projet de manière précise et comportant toutes les pièces permettant l’instruction du dossier, réputé complet, par les services de la Région. Après examen du dossier, la demande de subvention sera soumise à la Commission permanente régionale pour délibération.  Le versement de la subvention est subordonné à la signature préalable d’une convention.'

In [24]:
rag_chain.invoke("Concernant l'aide 'Plan route de demain : Électromobilité', quelles sont les démarches à faire ?")

'L\'aide "Plan route de demain : Électromobilité" vise à encourager l\'utilisation de véhicules électriques en offrant des incitations financières et des conseils aux clients intéressés par cette transition. Les démarches à suivre pour bénéficier de cette aide sont les suivantes :\n\n1. Renseignements sur les véhicules électriques : Le client doit d\'abord se renseigner sur les différents modèles de véhicules électriques disponibles sur le marché, ainsi que sur leurs caractéristiques techniques, leur autonomie, et les avantages et inconvénients de leur utilisation. Il peut consulter des sites spécialisés, des forums ou contacter des experts en électromobilité pour obtenir ces informations.\n\n2. Évaluation des besoins et des contraintes : Le client doit ensuite évaluer ses besoins en matière de mobilité et les contraintes auxquelles il pourrait être confronté lors de l\'utilisation d\'un véhicule électrique. Il doit prendre en compte des facteurs tels que la distance quotidienne parcou